In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd


In [11]:
con = pymysql.connect(host='localhost', user='root', password='1234',
                      db='MySQl', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True, # 결과 DB 반영 (Insert or update)
                      cursorclass=pymysql.cursors.DictCursor # DB조회시 컬럼명을 동시에 보여줌
                     )
cur = con.cursor()

In [12]:
sql = "SELECT * FROM trad" # customers 테이블 전체를 불러옴
cur.execute(sql)
rows = cur.fetchall()
con.close() # DB 연결 종료
print(rows)

ProgrammingError: (1146, "Table 'mysql.trad' doesn't exist")